In [1]:
from keplergl import KeplerGl
import pandas as pd

In [35]:
'''
Define paths and parameters
'''
DATA_PATH = './17_18.csv'

GEO_DATA_PATH = './country_lat_long.csv'

COUNTRIES = ['nl', 'lu', 'fr', 'ch', 'au', 'cz', 'pl', 'dk', 'sw']

### Data for 1 Day
RESAMPLE_INTERVAL = '1H'
TIMESPAN = (pd.Timestamp(2017, 1, 1, 0),pd.Timestamp(2017, 1, 2, 0))

### Data for one week
#RESAMPLE_INTERVAL = '3H'
#TIMESPAN = (pd.Timestamp(2017, 1, 1, 0),pd.Timestamp(2017, 1, 7, 0))

### Data for one month
#RESAMPLE_INTERVAL = '24H'
#TIMESPAN = (pd.Timestamp(2017, 1, 1, 0),pd.Timestamp(2017, 2, 1, 0))

### Data for one year
#RESAMPLE_INTERVAL = '168H'
#TIMESPAN = (pd.Timestamp(2017, 1, 1, 0),pd.Timestamp(2018, 1, 1, 0))

In [36]:
# load dataset
DATASET = pd.read_csv(DATA_PATH, sep=';')

# replace Datum Uhrzeit with proper timestamp as index
DATASET['timestamp'] = DATASET.Datum.map(str) + ' ' + DATASET.Uhrzeit
DATASET = DATASET.drop(['Uhrzeit', 'Datum'], axis=1)
DATASET.set_index('timestamp', inplace=True)
# transform to time series
DATASET.index = pd.to_datetime(DATASET.index)
DATASET = DATASET.apply(pd.to_numeric, errors='coerce')
DATASET.fillna(0, inplace=True)

# load geo data
GEO_DATA = pd.read_csv(GEO_DATA_PATH, sep=';')
print(GEO_DATA)

       country       long        lat
0    nl_import   5.387827  52.156113
1    nl_export   6.387827  52.156113
2    lu_import   6.103210  49.745998
3    lu_export   7.103210  49.745998
4    fr_import   2.398782  47.081013
5    fr_export   3.398782  47.081013
6    ch_import   8.400650  46.819950
7    ch_export   9.400650  46.819950
8    au_import  14.322770  47.554008
9    au_export  15.322770  47.554008
10   cz_import  15.781199  50.034309
11   cz_export  16.781199  50.034309
12   pl_import  19.455982  51.759247
13   pl_export  20.455982  51.759247
14   dk_import   8.973180  56.138481
15   dk_export   9.973180  56.138481
16   sw_import  15.621373  58.410809
17   sw_export  16.621373  58.410809
18  net_export   9.494630  51.318729


In [37]:
'''
Define data operation functions
'''

def shrinkIntervalMean(data, interval):
    '''
    returns the resampled dataset with the mean of the timestamp values in the given interval, 
    e.g. '10H' for 10 hour intervals
    '''
    return data.resample(interval).mean()

def toKeplerData(dataset, geodata):
    '''
    returns the given time series with each column as a distinct timestamp 
    in the series with the specific lat/lang attached
    '''
    series = pd.DataFrame()
    # columns = list(dataset.columns)
    for index, row in dataset.iterrows():
        # not nice but efficient
        if index == TIMESPAN[1]:
            return series
        for country in COUNTRIES:
            geo_ex = geodata.loc[geodata['country'] == country + '_export']
            geo_im = geodata.loc[geodata['country'] == country + '_import']
            newTimestamp = pd.DataFrame({'value_im':row[country + '_import'],'lat_im':geo_im.iloc[0]['lat'],
                                         'long_im':geo_im.iloc[0]['long'],
                                        'value_ex':row[country + '_export'],'lat_ex':geo_ex.iloc[0]['lat'],
                                         'long_ex':geo_ex.iloc[0]['long']},
                                        index=[index])
            
            series = series.append(newTimestamp)
            
        net_geo = geodata.loc[geodata['country'] == 'net_export'].iloc[0]
        
        # Include data for net export
        if row['net_export'] > 0:    
            series = series.append(pd.DataFrame({'value_im':0,'lat_im':net_geo['lat'],
                                         'long_im':net_geo['long'],
                                        'value_ex':row['net_export'],'lat_ex':net_geo['lat'],
                                         'long_ex':net_geo['long']},
                                        index=[index]))
        else:
            series = series.append(pd.DataFrame({'value_im':row['net_export'],'lat_im':net_geo['lat'],
                                         'long_im':net_geo['long'],
                                        'value_ex':0,'lat_ex':net_geo['lat'],
                                         'long_ex':net_geo['long']},
                                        index=[index]))


    return series

'''
Test functions
'''
#DATASET = shrinkTimeGran(DATASET, '5H')
#DATASET = shrinkIntervalMean(DATASET, '8H')
#print(toKeplerData(DATASET, GEO_DATA).head(5))

'\nTest functions\n'

In [38]:
'''
Data preparation
'''

# reduce time series granularity to 1 Day
DATASET = shrinkIntervalMean(DATASET, RESAMPLE_INTERVAL)
series = toKeplerData(DATASET, GEO_DATA)

'''
Export data to csv
'''
series.to_csv(r'geo_data_' + RESAMPLE_INTERVAL +'.csv')

In [39]:
'''
Import data from csv and transform it for map
'''

map_data = pd.read_csv('./geo_data_' + RESAMPLE_INTERVAL +'.csv', sep=",", parse_dates=[0], index_col=0)[TIMESPAN[0]:TIMESPAN[1]]
map_data['time'] = map_data.index.strftime('%Y-%m-%d %H:%M:%S')
map_data.set_index('time', inplace=True, drop=False)

In [7]:
'''
Create map
'''

german_electricity_map = KeplerGl(height=500)
german_electricity_map.add_data(data=map_data, name='Strom Im-/Export')
german_electricity_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Strom Im-/Export': {'index': ['2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:…

In [8]:
'''
Save config and save map as html
'''

map_config = german_electricity_map.config
german_electricity_map.save_to_html(file_name='german_electricity_map_' + RESAMPLE_INTERVAL + '.html')


Map saved to german_electricity_map_168H.html!


In [40]:
'''
Create new map with same config
'''

german_electricity_map_2 = KeplerGl(height=500, data={"Strom Im-/Export": map_data}, config=map_config)
german_electricity_map_2

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'Strom Im-/Export', 'id': 'jg…

In [41]:
german_electricity_map_2.save_to_html(file_name='german_electricity_map_' + RESAMPLE_INTERVAL + '.html')

Map saved to german_electricity_map_1H.html!
